In [1]:
import random
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set, Tuple

import networkx as nx
import numpy as np
import pandas as pd

from icm_diffusion import simulate_diffusion_ICM
from src.graph import (
    create_polarized_graph,
    graph_loader,
    random_color_graph,
    spectral_bipartition_coloring,
)
from src.seed import seed_degree, seed_random, seed_mia


In [2]:
from edge_addition import (
    edge_addition_custom,
    edge_addition_adamic_adar,
    edge_addition_preferential_attachment,
    edge_addition_jaccard,
    edge_addition_degree,
    edge_addition_topk,
    edge_addition_prob,
    edge_addition_kkt,
    edge_addition_random,
)

In [3]:
# Function to evaluate and compare the graph modifications
def evaluate_graph_modifications(G, seeds, k, max_iter, budget):
    # Simulate diffusion on the original graph
    count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G, seeds, max_iter)
    
    # Results for the original graph
    original_results = pd.DataFrame({
        'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
        'Original Graph': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
    })

    # Get the number of nodes and edges for all graphs
    graph_info = {
        'Metric': ['Number of Nodes', 'Number of Edges'],
        'Original Graph': [G.number_of_nodes(), G.number_of_edges()],
    }
    
    # Define a list of modification functions
    modification_functions = {
        'Adamic Adar': edge_addition_adamic_adar,
        'PrefAtt': edge_addition_preferential_attachment,
        'Jaccard': edge_addition_jaccard,
        'Degree': edge_addition_degree,
        'TopK': edge_addition_topk,
        'Prob': edge_addition_prob,
        'KKT': edge_addition_kkt,
        'Random': edge_addition_random,
        'Custom': edge_addition_custom
    }

    combined_results = original_results.copy()

    # Evaluate each graph modification
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k, budget)
        count, count_std, color_count, color_count_std = simulate_diffusion_ICM(modified_graph, seeds, max_iter)

        adapted_results = pd.DataFrame({
            'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
            f'Adapted Graph {method_name}': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
        })

        combined_results = pd.merge(combined_results, adapted_results, on='Metric')
        graph_info[f'Adapted Graph {method_name}'] = [modified_graph.number_of_nodes(), modified_graph.number_of_edges()]
    
    
    graph_info_df = pd.DataFrame(graph_info)

    # Combine all results into one DataFrame
    final_results = pd.concat([graph_info_df, combined_results], ignore_index=True)

    # Transpose the DataFrame and set the first row as the header
    final_results = final_results.T
    final_results.columns = final_results.iloc[0]  # Set the first row as the column names
    final_results = final_results.drop(final_results.index[0])  # Drop the first row

    return final_results

In [4]:
#G = create_polarized_graph(1000, 0.2, 0.01)
G = graph_loader('datasets/congress_network/congress.edgelist')
#color the graph
spectral_bipartition_coloring(G)

Number of Nodes: 475
Number of Edges: 13289


/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:233: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [5]:
seed = seed_mia(G, 100)
k = 15
max_iter = 1000
budget = 3 * len(seed)
final_results = evaluate_graph_modifications(G, seed, k, max_iter, budget)
final_results

100%|██████████| 1000/1000 [00:02<00:00, 403.40it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,475.0,13289.0,223.846,46.736,27.246,14.709
Adapted Graph Adamic Adar,475.0,13577.0,223.943,48.919,29.186,16.467
Adapted Graph PrefAtt,475.0,13589.0,226.877,49.078,29.431,16.096
Adapted Graph Jaccard,475.0,13589.0,260.746,42.346,43.404,15.81
Adapted Graph Degree,475.0,13589.0,229.33,47.038,30.966,16.219
Adapted Graph TopK,475.0,13589.0,220.167,48.153,27.771,15.767
Adapted Graph Prob,475.0,13589.0,295.133,38.453,66.019,15.618
Adapted Graph KKT,475.0,13589.0,223.473,48.84,28.62,15.543
Adapted Graph Random,475.0,13589.0,237.598,45.881,46.366,18.686
Adapted Graph Custom,475.0,13589.0,230.502,50.211,36.455,19.184


In [6]:
seed = seed_random(G, 100)
k = 15
max_iter = 1000
budget = 3 * len(seed)
final_results = evaluate_graph_modifications(G, seed, k, max_iter, budget)
final_results

100%|██████████| 1000/1000 [00:02<00:00, 365.07it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,475.0,13289.0,273.215,44.853,35.015,15.711
Adapted Graph Adamic Adar,475.0,13572.0,274.811,43.757,36.822,16.138
Adapted Graph PrefAtt,475.0,13589.0,276.964,44.513,39.634,17.643
Adapted Graph Jaccard,475.0,13589.0,301.72,39.176,51.804,16.29
Adapted Graph Degree,475.0,13589.0,276.901,43.136,38.799,16.753
Adapted Graph TopK,475.0,13589.0,275.835,44.82,36.715,16.291
Adapted Graph Prob,475.0,13589.0,306.306,40.039,50.532,17.214
Adapted Graph KKT,475.0,13589.0,273.881,44.512,37.393,16.911
Adapted Graph Random,475.0,13589.0,281.088,45.082,47.644,18.412
Adapted Graph Custom,475.0,13589.0,275.879,45.062,40.211,18.661
